In [9]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
class ExpDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values.astype(np.float32)
        self.labels = self.data.iloc[:, -1].values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.features[idx].reshape(64, 64)  # 64x64 images
        label = self.labels[idx]
        
        image = torch.from_numpy(image).unsqueeze(0)  # Add channel dimension
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

In [11]:
# Load the datasets
train_data = ExpDataset('train_data.csv')
val_data = ExpDataset('val_data.csv')
test_data = ExpDataset('test_data.csv')

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [12]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [13]:
num_classes = 19  # 0-9, add, dec, div, eq, mul, sub, x, y, z
model = CNN(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
 # Training setup
patience = 3
min_delta = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)
num_epochs = 100
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

best_val_loss = float('inf')
epochs_without_improvement = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        train_total += target.size(0)
        train_correct += predicted.eq(target).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            val_total += target.size(0)
            val_correct += predicted.eq(target).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = val_correct / val_total
    
    scheduler.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    print(f'Learning Rate: {scheduler.get_last_lr()[0]:.6f}')
    
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), 'normalized_model.pth')
    else:
        epochs_without_improvement += 1
    
    if epochs_without_improvement >= patience:
        print(f'Early stopping triggered after {epoch+1} epochs')
        break
    
    print('---')

Epoch 2/100, Batch 100/101, Loss: 0.3185
Epoch 2/100:
Train Loss: 0.7018, Train Accuracy: 0.7782
Val Loss: 0.4042, Val Accuracy: 0.8727
Learning Rate: 0.001000
---
Epoch 3/100, Batch 0/101, Loss: 0.3695
Epoch 3/100, Batch 100/101, Loss: 0.4567
Epoch 3/100:
Train Loss: 0.4543, Train Accuracy: 0.8561
Val Loss: 0.3283, Val Accuracy: 0.9075
Learning Rate: 0.001000
---
Epoch 4/100, Batch 0/101, Loss: 0.3796
Epoch 4/100, Batch 100/101, Loss: 0.1928
Epoch 4/100:
Train Loss: 0.3494, Train Accuracy: 0.8839
Val Loss: 0.2219, Val Accuracy: 0.9385
Learning Rate: 0.001000
---
Epoch 5/100, Batch 0/101, Loss: 0.3038
Epoch 5/100, Batch 100/101, Loss: 0.1901
Epoch 5/100:
Train Loss: 0.2986, Train Accuracy: 0.9022
Val Loss: 0.2520, Val Accuracy: 0.9373
Learning Rate: 0.000100
---
Epoch 6/100, Batch 0/101, Loss: 0.1610
Epoch 6/100, Batch 100/101, Loss: 0.1918
Epoch 6/100:
Train Loss: 0.1801, Train Accuracy: 0.9444
Val Loss: 0.1557, Val Accuracy: 0.9634
Learning Rate: 0.000100
---
Epoch 7/100, Batch 0/101

In [15]:
# Load the best model for testing
model.load_state_dict(torch.load('normalized_model.pth'))

# Test the model
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

class_mapping = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'add', 11: 'dec', 12: 'div', 13: 'eq', 14: 'mul', 15: 'sub', 16: 'x', 17: 'y', 18: 'z'
}

sample_predictions = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        _, predicted = output.max(1)
        test_total += target.size(0)
        test_correct += predicted.eq(target).sum().item()
        
        for true, pred in zip(target[:5], predicted[:5]):
            sample_predictions.append((true.item(), pred.item()))
        
        if len(sample_predictions) >= 20:
            break

test_loss /= len(test_loader)
test_accuracy = test_correct / test_total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

print("\nSample Predictions:")
for i, (true, pred) in enumerate(sample_predictions[:20], 1):
    print(f"Sample {i}:")
    print(f"Ground Truth: {class_mapping[true]}")
    print(f"Prediction: {class_mapping[pred]}")
    print(f"Correct: {true == pred}")
    print("---")

Test Loss: 0.0134, Test Accuracy: 0.9727

Sample Predictions:
Sample 1:
Ground Truth: mul
Prediction: mul
Correct: True
---
Sample 2:
Ground Truth: 4
Prediction: 4
Correct: True
---
Sample 3:
Ground Truth: eq
Prediction: eq
Correct: True
---
Sample 4:
Ground Truth: 7
Prediction: 7
Correct: True
---
Sample 5:
Ground Truth: sub
Prediction: sub
Correct: True
---
Sample 6:
Ground Truth: add
Prediction: add
Correct: True
---
Sample 7:
Ground Truth: 9
Prediction: 9
Correct: True
---
Sample 8:
Ground Truth: dec
Prediction: dec
Correct: True
---
Sample 9:
Ground Truth: mul
Prediction: mul
Correct: True
---
Sample 10:
Ground Truth: 3
Prediction: 3
Correct: True
---
Sample 11:
Ground Truth: 1
Prediction: 1
Correct: True
---
Sample 12:
Ground Truth: 0
Prediction: 8
Correct: False
---
Sample 13:
Ground Truth: 7
Prediction: 4
Correct: False
---
Sample 14:
Ground Truth: 2
Prediction: 2
Correct: True
---
Sample 15:
Ground Truth: 2
Prediction: 2
Correct: True
---
Sample 16:
Ground Truth: 6
Prediction: